In [0]:
# Run this cell to authenticate yourself to BigQuery.
from google.colab import auth
auth.authenticate_user()
project_id = "project-223804"

In [0]:
# Some imports you will need
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import altair as alt
%matplotlib inline
plt.style.use('seaborn-whitegrid')
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

# Analysis of your dataset

I will be working with the New York Taxi trip dataset. It contains information about taxi trips on 3 kinds of cabs, limousines, green cabs and yellow cabs, from a period of 2014 to 2018.

**Tables and Schema**
The taxi dataset consists of 4 main types of tables. 3 of them correspond to the 3 different kinds of cabs, and the last corresponds to information about differnet kinds of taxi zones.

*Italics* refers to keys

For limousine cabs, the table names are tlc_fhv_trips_20XX, where XX can be 15, 16 or 17. It's schema is:
For 2015 and 2016,
(location_id, pickup_datetime, dispatching_base_num, borough, zone, service_zone).
For 2017,
(location_id, pickup_datetime, dispatching_base_num, borough, zone, service_zone, dropoff_datetime, dropoff_location_id, dropoff_borough, dropoff_zone, dropoff_service_zone)
The number of rows is 63867609 for 2015, 133285141 for 2016, 192092698 for 2017.

For green cabs, the table names are tlc_green_trips_20XX, where XX can be 14, 15, 16, 17, 18. It's schema is:
(vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, pickup_longtitude, pickup_latitude, ...)
The number of rows is 15837001 for 2014, 19233765 for 2015, 16385532 for 2016, 11740667 for 2017, 4737308 for 2018

For yellow cabs, the table names are tlc_yellow_trips_20XX, where XX can be 15, 16, 17, or 18. It's schema is similar to that for green cabs (the information provided per trip is similar), with a pertinent difference being that green cabs have an added column describing the trip type, while yelllow cabs do not. This is because yellow cabs in New York are specifically street hail only, while green cabs can take either street hail or e-bookings.
The number of rows is 146112989 for 2015, 131165043 for 2016, 113496874 for 2017, 63356111 for 2018

For these 3 tables, my conjecture is that they do not have key values, as I have observed at least one duplicate row in one of the tables and have not been able to find a clear column that could serve as a key for a trip (something like a trip_id). For example, observing rows 1 and 2 tlc_fhv_trips_2015, they are identical rows. In all the tables, there is information such as pickup time and dropoff time, but these are not unique to a trip; while it is extremely unlikely that two trips match on the whole series of columns, it is not impossible. We know that tlc_fhv_trips_2015 has duplicate rows, but the other trips might not, and in the likely possibility that they do not actually have any duplicate rows, then the entire row serves as a key. We can verify this using a query that groups on every column, and returns a row when it has a count more than 1. If the query is non empty, then there are no duplicates, and we know that the full set of columns on every row is a key for itself, else, we know there are duplicates and there are no keys. The exact query is here: https://stackoverflow.com/questions/306743/how-to-detect-duplicate-rows-in-a-sql-server-table. Due to the large size of the dataset, we will not be doing this check for every single table to see if there is a key, but we are assume that since there are so many columns and the chance of a collision is so small, that there are no duplicates and each row is a key for itself.

There is one table type, tlc_taxi_zones, for taxi zones. It's schema is:
(*location_id*, zone, borough, geom)
The location_id identifies a specific location, which can be characterized by the zone and burrough it is in, and the set of geometrical coordinates it corresponds to.

# Exploring your questions, with appropriate visualizations

We are interested in answering one general questions:
What is a good model to predict the price of a cab ride?

For 1, the question is relevant because Uber alreay has a pricing algorithm, one can still take a metered cab that is not under Uber, and it may be valuable to provide price prediction for those. Furthermore, this can be seen as a simulation of a pricing prediction that Uber might be using themselves.
For 2, the question is relevant because if we can define a reasonable metric to measure the value of a cab ride for cab drivers, and then find predictors for that, this can help cab drivers maximize the value of their trips.

To do so, we will first undertake a data vsiualization task in order to visualize trends that might suggest a relationship we can leverage in our model.

Before that, we will define which tables we want to be using. Looking at the tables for limousine cabs, the 2015 and 2016 data sets do not have any dropoff data. Our project intends to examine the hypothesis of how year of travel is going to affect the cost of a ride, and therefore, we have chosen to omit limousine cabs from consideration, as without 2015 and 2016, we only have 2017 as a year.
Furthermore, we will be focusing the yellow cabs, as they are an iconic symbol of New York, and we cannot consider both yellow and green cabs together because having e-bookings vs hail-only services will cause the fares to be different, meaning they cannot be considered equivalently. Furthermore, it is not difficult to extend by a single feature for the model we have for yellow cabs to green cabs, so it may be redundant to do both. We will simply do a model for yellow cabs.

We hypothesize that the following features will have an effect on the prices, with some justification for them:
*   Year in which the trip was made (because cab fares might change with inflation, etc.)
*   Month in which the trip was made (because cab fares may have seasonal patterns)
*   Day of the week in which the trip was made (Work days may have different demand levels from weekends, for example)
*   Time in which the trip was made (Peak hours are very different in cost from off peak hours)
*   Trip distance (obviously, since cabs are metered)
*   Trip duration (Presumably, longer trips indicate longer distance, and may indicate trips taken at certain times, etc.)
*   Place of pickup (Some links may be start and end location determine the routes taken, which may affect toll rates, and different locations mean different customer types who want to travel to different areas)
*   Place of dropoff (For example, where a person drops off might indicate where the cab was taken from, which may influence the fare price. Airports may have trips from touristy parts of the city, that usually have a range of distances from the airport)
*   Passenger count (Cabs may charge for more people)
*   Type of trip, as indicated by rate_code (rate_code column in the taxi trip table indicates the type of trip: 1= Standard rate 2=JFK 3=Newark 4=Nassau or Westchester 5=Negotiated fare 6=Group ride. Logically, these will have different rates)
*   Weather of pickup area (Different weather patterns affect traffic, routes, time, demand, etc.)
*   Weather of dropoff area (Same reasons as pickup area)

We will first test the trend in cab fares by year, by doing a simple query on average cab fare by year, across the different years (2015, 2016, 2017, 2018). Note that rand() is a function that we put in place to ensure that the sampling is drawn randomly without regard to any order, so as to avoid unintended bias, such as accidentally preferring trips from earlier dates, for example.

In [0]:
#Query to obtain average cab fare for 2015

%%bigquery --project $project_id v1.1

SELECT AVG(total_amount) as avg_2015
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
WHERE RAND() < 100000/146112989 AND total_amount >  0

,avg_2015
0,16.067266


In [0]:
#Query to obtain average cab fare for 2016

%%bigquery --project $project_id v1.2

SELECT AVG(total_amount) as avg_2016
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2016`
WHERE RAND() < 100000/131165043 AND total_amount >  0

,avg_2016
0,16.307496


In [0]:
#Query to obtain average cab fare for 2017

%%bigquery --project $project_id v1.3

SELECT AVG(total_amount) as avg_2017
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2017`
WHERE RAND() < 100000/113496874 AND total_amount >  0

#Error: Invalid numeric encoding; while executing the filter on column 'total_amount'; File: ':mdb=cloud-dataengine'
#Basically, The 2017 data contains a great number of rows with INVALID under the column for total_amount.
#I troubleshooted with TA Amelia, using CAST, SAFE_CAST, IS NOT NULL, in several permutations, but was not able to arrive at a solution.
#I checked with TA Amelia and TA Alan, and they said that it was OK if I explained my error and mentioned that I will not be using year as a feature

In [0]:
#Query to obtain average cab fare for 2018

%%bigquery --project $project_id v1.4

SELECT AVG(total_amount) as avg_2018
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018`
WHERE RAND() < 100000/63356111 AND total_amount >  0

#KeyError: 'NUMERIC'
#The code runs in Bigquery and returns 16.085041538.

Refer to technical error message above for 2017. From query, 2015, 2016 and 2018 has average fare values 16.067266, 16.307496 and 16.085041538.

There doesn't seem to be a strong consistent relationship between year and fare, and the number of years we have is extremely limited. Coupled with the technical difficulties with 2017 data, we will be skipping the analysis of year as a feature.

We will now study the other features in the context of a single year (2016). We will first clean the data by sampling 10,000,000 rows from it, in order to facilitate other queries that we will be doing. These rows we will then filter to make sure we have a clean dataset. We want to consider a conventional trip made; people get in a cab and actually travel a certain distance and time to a place. If someone gets in a cab, the trip is recorded but he leaves in 30 seconds as a mistake, for example, we do not want to consider that trip. Also, some complete trips may have incomplete entries, but to facilitate analaysis, we will pick only complete ones, and make the weak assumption that this has a negligible impact on the analysis.
* Valid datetimes (NOT NULL)
* Valid distance (> 0)
* Valid duration (>= 0, a trip could conceivably be less than a minute, for example)
* Valid locations (NOT NULL)
* Valid passenger count (> 0)
* Valid rate type (NOT NULL, from 1 to 6)
* Valid weather data (to be considered later)
* Valid payment amount

In [0]:
#Query run in BigQuery to build my cleaned dataset for yellow trips in 2016 (not to be run here)

SELECT *
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2016`
WHERE RAND() < 10000000/131165043 AND pickup_datetime IS NOT NULL AND dropoff_datetime IS NOT NULL AND trip_distance > 0 AND pickup_longitude IS NOT NULL AND pickup_latitude IS NOT NULL
      AND passenger_count > 0 AND rate_code IS NOT NULL AND rate_code > 0 AND rate_code < 7 AND dropoff_longitude IS NOT NULL AND dropoff_latitude IS NOT NULL AND total_amount > 0
  
  #Table was saved in `project-223804.MyTaxiDataset.yellowtrips_2016_cleaned`

What we eventually want to study for our dataset is the relationship between the weather at the time and place of a pickup/dropoff and the fare of the trip associated with that location. However, there is no weather table in the taxi dataset. We will do an extensive engineering of this feature, as weather is well known to affect cab rates and it is valuable to study this hypothesis.

Each trip takes place in a zone, which is in a borough. We assume that weather is relatively constant across a borough (If it is raining in one part of Manhattan, it is raining in another). What we are going to do is to look at the noaa_gsod dataset. It basically contains daily readings of several weather metrics, for each weather station. For every zone, we will identify weather stations that is within 1.2km of any point in the zone, which means that they are within 1.2km of any point in the borough the zone is in, and then join these rows with the weather entry table, and group the weather entry data rows by boroughs, year, month and date, and then average over all the weather data of all the active weather stations within the each of these groupings to obtain an estimate for the weather at a particular time and borough. The weather data, however, is extremeley messy and this is a complicated process.

Firstly, noaa_gsod contains stations, a table containing key information about each weather station, such as it's longitude and latitude. A problem with this dataset is that it is linked to gsod2016, the weather table, via a key called usaf, which is like an id for the station. However, several stations are unlabelled (they are default assigned a value 99999), meaning that usaf cannot uniquely identify a station. This means that when we associate every zone and thus borough with a collection of weather stations, we have several 99999 stations associated with each borough, and eventually when we are trying to match a weather entry to a borough at a particular time, there are several 99999 stations with the same period, and these 99999 stations could be anywhere (eg. Honduras, because 99999 is just a bucket for unlabelled weather stations). This mean that our borough data will take weather data from outside a borough.

Furthermore, we want to have weather stations in the right time period (whole of 2016).

1.2km was chosen because it was a nice figure that ensured that every single borough would have at least one fully active weather station over the entire time period.

In [0]:
#Query run in BigQuery to clean stations data and join it with tlc_taxi_zones. Returns several rows that tells us a borough - weather station pairing.

SELECT borough, usaf
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` as zones, `bigquery-public-data.noaa_gsod.stations` AS stations
WHERE ST_DISTANCE(zones.zone_geom, ST_GEOGPOINT(stations.lon, stations.lat)) < 1200 AND EXTRACT(YEAR FROM PARSE_DATE('%Y%m%d', stations.begin)) < 2016
      AND EXTRACT(YEAR FROM PARSE_DATE('%Y%m%d', stations.end)) > 2016 AND usaf <> '999999'
GROUP BY borough, usaf

#We have six boroughs because EWR is also considered as a borough in the context of this data.
#Saved in `project-223804:MyTaxiDataset.cleaned_station_borough`

In the below query, the calculations for the mean temp, snow, precipitation, etc. were made because 9 series values are used in place of empty entries, For temperature, to get the real mean, we take the sum over all values, subtract away the sum of 9 series values to get a reduced sum, and then divide it by a reduced count reflecting the actual number of valid temperature measurements.

For precipitation and snow data, according to the table description, a 9 series data usually indicates the a 0 value; if there is no snow, you put 999.9. Hence, what you want to do is to do a reduced sum, but divide it over a non reduced count, because 9 series data are not assumed to be invalid but rather actually 0 entries.

Also, there are 6 borroughs, so there should be 5 x 365 = 2190 entries. Indeed, we have 2194 rows, which indicates this data is complete.

In [0]:
#Query run in Bigquery to join the weather data with borough, obtaining weather values for every borough-period set

SELECT b_data.borough as borough, year, mo, da, (SUM(temp)-(COUNTIF(temp = 9999.9)*9999.9))/(COUNT(*)-COUNTIF(temp = 9999.9)) AS mean_temp,
       (SUM(prcp)-(COUNTIF(prcp = 99.9)*99.9))/COUNT(*) AS mean_precipitation,
       (SUM(sndp)-(COUNTIF(sndp = 999.9)*999.9))/COUNT(*) AS mean_snowdepth
FROM `project-223804.MyTaxiDataset.cleaned_station_borough` AS b_data, `bigquery-public-data.noaa_gsod.gsod2016` AS w_data
WHERE b_data.usaf = w_data.stn AND temp IS NOT NULL AND prcp IS NOT NULL AND sndp IS NOT NULL
GROUP BY b_data.borough, year, mo, da

#This is saved in `project-223804.MyTaxiDataset.weather_borough_period_2016`

Now, we are going to build a large table that allows us to run our queries with ease.

We want to get:
* pickup hour
* pickup day of week
* pickup day
* pickup month
* dropoff hour
* dropoff day
* dropoff month
* trip duration
* trip distance
* pickup longitude
* pickup latitude
* dropoff longitude
* dropoff latitude
* pickup zone
* dropoff zone
* pickup + dropoff zone
* pickup_borough
* dropoff_borough
* passenger count
* rate_code
* fare amount
* weather data

Note that we clean our data by restricting trip duration to below 400 minutes, as during an earlier run of the project, we detected the following anomaly, which we have screenshotted and presented below:


![](https://drive.google.com/uc?export=view&id=1zJK1-89FwYU1MoGG2K20EP_1GYoxGyfX)

In [0]:
#Query run in BigQuery to build our sample of 400,000 pre joining with weather

SELECT TIME_TRUNC(CAST(pickup_datetime AS TIME), HOUR) AS pickup_hour, EXTRACT(DAYOFWEEK FROM CAST(pickup_datetime AS DATE)) AS pickup_dayofweek,
       EXTRACT(DAY FROM CAST(pickup_datetime AS DATE)) AS pickup_day,
       EXTRACT(MONTH FROM CAST(pickup_datetime AS DATE)) AS pickup_month, TIME_TRUNC(CAST(dropoff_datetime AS TIME), HOUR) AS dropoff_hour,
       EXTRACT(DAY FROM CAST(dropoff_datetime AS DATE)) AS dropoff_day, EXTRACT(MONTH FROM CAST(dropoff_datetime AS DATE)) AS dropoff_month,
       DATETIME_DIFF(dropoff_datetime, pickup_datetime, MINUTE) as trip_duration, trip_distance, pickup_longitude, pickup_latitude,
       dropoff_longitude, dropoff_latitude, pickup_zones.zone_name AS pickup_zone, dropoff_zones.zone_name AS dropoff_zone,
       CONCAT(pickup_zones.zone_name, "-", dropoff_zones.zone_name) AS pickup_dropoff,
       pickup_zones.borough AS pickup_borough, dropoff_zones.borough AS dropoff_borough, passenger_count, rate_code, total_amount
       
FROM  (SELECT data.*, ST_GEOGPOINT(pickup_longitude, pickup_latitude) AS pickup_point, ST_GEOGPOINT(dropoff_longitude, dropoff_latitude) AS dropoff_point
      FROM `project-223804.MyTaxiDataset.yellowtrips_2016_cleaned` AS data
      WHERE DATETIME_DIFF(dropoff_datetime, pickup_datetime, MINUTE) < 400 AND DATETIME_DIFF(dropoff_datetime, pickup_datetime, MINUTE) >= 0
      ORDER BY rand()
      LIMIT 400000) AS trips, `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS pickup_zones,
      `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS dropoff_zones
      
WHERE ST_CONTAINS(pickup_zones.zone_geom, pickup_point) AND ST_CONTAINS(dropoff_zones.zone_geom, dropoff_point)

#TABLE was saved in `project-223804.MyTaxiDataset.sample_preweather`

In [0]:
#Query run in BigQuery to add weather data to joined data set.

SELECT preweather.*, weather_data1.mean_temp AS pickup_temp, weather_data1.mean_precipitation AS pickup_prcp, weather_data1.mean_snowdepth AS pickup_sndp,
       weather_data2.mean_temp AS dropoff_temp, weather_data2.mean_precipitation AS dropoff_prcp, weather_data2.mean_snowdepth AS dropoff_sndp
FROM `project-223804.MyTaxiDataset.sample_preweather` AS preweather, `project-223804.MyTaxiDataset.weather_borough_period_2016` AS weather_data1,
     `project-223804.MyTaxiDataset.weather_borough_period_2016` AS weather_data2
WHERE preweather.pickup_borough = weather_data1.borough AND preweather.pickup_month = CAST(weather_data1.mo AS INT64) AND preweather.pickup_day = CAST(weather_data1.da AS INT64) AND
      preweather.dropoff_borough = weather_data2.borough AND preweather.dropoff_month = CAST(weather_data2.mo AS INT64) AND preweather.dropoff_day = CAST(weather_data2.da AS INT64)
  
#TABLE was saved in `project-223804.MyTaxiDataset.sample_postweather`

`project-223804.MyTaxiDataset.sample_postweather` is our final table, which contains all the features we want to study and will be used for both visualization and ML training, evaluation and prediction task (we will be using part of 2016 trips to predict the other part of 2016 trips). **Note that this table only has entries from the first 6 months of 2016, because only those months have the location data. We will continue to use this dataset, as it is the only section of data available that contains all features we want to examine (half of 2015 has n rate code, the other half of 2016 has no location data)**

We will begin with the visualization task. The first features we want to study are how the pickup timing, days and months , and dropoff timings affect the taxi fare. Dropoff days and months are omitted because they are likely to almost always be identical to pickup days and months.

In [8]:
#Query for pickup timing vs. fare amount

%%bigquery --project $project_id v2

SELECT CAST(EXTRACT(HOUR FROM pickup_hour) AS INT64) AS hour, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY pickup_hour

,hour,average_fare
0,20,15.393952
1,9,14.942899
2,2,15.623836
3,15,16.611156
4,16,17.261609
5,5,19.872478
6,21,15.844144
7,4,18.276282
8,12,15.510872
9,22,16.462393


In [0]:
alt.Chart(v2).mark_bar().encode(
x='hour',
y=alt.Y('average_fare', scale=alt.Scale(type='linear'))
)

Chart({
  data:     hour  average_fare
  0     20     15.393952
  1      9     14.942899
  2      2     15.623836
  3     15     16.611156
  4     16     17.261609
  5      5     19.872478
  6     21     15.844144
  7      4     18.276282
  8     12     15.510872
  9     22     16.462393
  10     1     15.898393
  11     3     15.915911
  12    17     16.773894
  13    19     15.354760
  14     6     16.033689
  15    18     15.731498
  16    23     17.077662
  17    11     15.345308
  18     0     16.498310
  19    13     16.057835
  20    10     15.215634
  21    14     16.458522
  22     8     14.986902
  23     7     14.677487,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'hour'
    }),
    y: Y({
      scale: Scale({
        type: 'linear'
      }),
      shorthand: 'average_fare'
    })
  }),
  mark: 'bar'
})

There aree three peak periods for fares: early morning (4-5), evening (18), and near midnight (23-0)

In [0]:
#Query for pickup dayofweek vs. fare amount

%%bigquery --project $project_id v3

SELECT pickup_dayofweek, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY pickup_dayofweek

In [0]:
alt.Chart(v3).mark_bar(size = 40).encode(
x='pickup_dayofweek',
y=alt.Y('average_fare', scale=alt.Scale(type='linear'))
)

Chart({
  data:    pickup_dayofweek  average_fare
  0                 1     15.875530
  1                 7     14.923771
  2                 3     16.079314
  3                 2     16.132260
  4                 6     16.137142
  5                 5     16.480501
  6                 4     16.145228,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'pickup_dayofweek'
    }),
    y: Y({
      scale: Scale({
        type: 'linear'
      }),
      shorthand: 'average_fare'
    })
  }),
  mark: MarkDef({
    size: 40,
    type: 'bar'
  })
})

Fares are relatively constant across the week, peaking on friday, and then dipping on Sunday.

In [0]:
#Query for pickup day vs. fare amount

%%bigquery --project $project_id v4

SELECT pickup_day, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY pickup_day

In [0]:
``alt.Chart(v4).mark_bar(size = 10).encode(
x='pickup_day',
y=alt.Y('average_fare', scale=alt.Scale(type='linear'))
)

Chart({
  data:     pickup_day  average_fare
  0           14     15.972761
  1           13     15.770058
  2           19     15.958472
  3           18     16.152825
  4           22     16.236850
  5           11     15.905181
  6           10     16.059460
  7           21     16.149199
  8            9     15.806226
  9            4     15.738874
  10           5     15.571205
  11          30     15.720353
  12           6     15.797493
  13           3     16.154193
  14          24     16.200830
  15           7     15.713577
  16          23     15.717842
  17          12     15.839074
  18          15     16.063087
  19          28     16.053295
  20          29     15.954368
  21          27     15.933465
  22           8     15.752319
  23          20     15.998638
  24          31     16.399523
  25           1     15.886223
  26          26     16.270345
  27          25     16.316220
  28           2     15.540554
  29          16     15.976906
  30          17     16.483998,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'pickup_day'
    }),
    y: Y({
      scale: Scale({
        type: 'linear'
      }),
      shorthand: 'average_fare'
    })
  }),
  mark: MarkDef({
    size: 10,
    type: 'bar'
  })
})

The day of the month in which a trip is taken has barely any influence on the average fare of the rides.

In [0]:
#Query for pickup month vs. fare amount

%%bigquery --project $project_id v5

SELECT pickup_month, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY pickup_month

In [0]:
alt.Chart(v5).mark_bar(size = 30).encode(
x='pickup_month',
y=alt.Y('average_fare', scale=alt.Scale(type='linear'))
)

Chart({
  data:    pickup_month  average_fare
  0             2     15.418225
  1             1     15.408677
  2             3     15.930010
  3             4     16.004723
  4             5     16.507876
  5             6     16.475425,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'pickup_month'
    }),
    y: Y({
      scale: Scale({
        type: 'linear'
      }),
      shorthand: 'average_fare'
    })
  }),
  mark: MarkDef({
    size: 30,
    type: 'bar'
  })
})

There are slight seasonal patterns in the data, with cab fares increasing with the month.

In [0]:
#Query for dropoff time vs. fare amount

%%bigquery --project $project_id v6

SELECT EXTRACT(HOUR FROM dropoff_hour) AS hour, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY dropoff_hour

In [0]:
alt.Chart(v6).mark_bar().encode(
x='hour',
y=alt.Y('average_fare', scale=alt.Scale(type='linear'))
)

Chart({
  data:     hour  average_fare
  0     20     15.556894
  1      9     15.173141
  2      2     16.005281
  3     15     16.348645
  4     16     17.140198
  5      5     19.305375
  6     21     15.795172
  7     10     15.107754
  8      4     17.262076
  9     12     15.286863
  10    22     16.183768
  11     1     16.753031
  12     3     16.070501
  13    17     16.788789
  14    19     15.855397
  15     6     15.369648
  16    18     16.355127
  17     0     17.378189
  18    11     15.214369
  19    13     15.687477
  20    23     17.209523
  21     7     14.035140
  22    14     15.901151
  23     8     14.571286,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'hour'
    }),
    y: Y({
      scale: Scale({
        type: 'linear'
      }),
      shorthand: 'average_fare'
    })
  }),
  mark: 'bar'
})

The influence of dropoff hour on average fare mirrors that of pickup hour, unsurprisingly

In [0]:
#Query for trip_duration vs. fare amount

%%bigquery --project $project_id v7

SELECT trip_duration, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY trip_duration

In [0]:
alt.Chart(v7).mark_circle(size = 15).encode(
alt.X('trip_duration', scale = alt.Scale(domain = (0, 130), clamp = True)),
y=alt.Y('average_fare', scale=alt.Scale(type='linear', domain = (0, 100), clamp = True))
)

Chart({
  data:      trip_duration  average_fare
  0              111     69.250000
  1              228    183.000000
  2               49     55.129848
  3               61     62.076467
  4               75     64.709149
  5               41     48.222596
  6               95     59.866364
  7               62     62.576529
  8               71     64.525464
  9               65     64.120196
  10              78     69.876522
  11               6      8.316423
  12              34     39.135707
  13              25     27.195491
  14              16     16.807386
  15              80     70.984750
  16              56     58.664212
  17             122     76.230000
  18             286     29.120000
  19             103     70.480000
  20             204    427.920000
  21              55     58.243127
  22              47     53.478368
  23              17     17.789556
  24              69     64.742087
  25             141     68.640000
  26              68     64.132685
  27               1      6.364568
  28             100     87.961000
  29              15     15.779502
  ..             ...           ...
  131             98     64.788000
  132            321     61.010000
  133             36     41.580771
  134             66     62.447333
  135             48     53.472020
  136            132     58.340000
  137            123     58.340000
  138            163     58.340000
  139            221     58.340000
  140            192     62.840000
  141            187     63.340000
  142            222     12.960000
  143             67     62.000500
  144             51     56.261716
  145              5      7.589583
  146             94     67.571176
  147            130     69.840000
  148            126     70.340000
  149            397     67.090000
  150            136    111.340000
  151            155    111.840000
  152            101     70.072000
  153             96     64.992308
  154             97     68.398182
  155             24     25.741700
  156              0     14.700721
  157             13     13.862620
  158             21     22.214494
  159              8      9.800689
  160             87     67.260800
  
  [161 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      scale: Scale({
        clamp: True,
        domain: (0, 130)
      }),
      shorthand: 'trip_duration'
    }),
    y: Y({
      scale: Scale({
        clamp: True,
        domain: (0, 100),
        type: 'linear'
      }),
      shorthand: 'average_fare'
    })
  }),
  mark: MarkDef({
    size: 15,
    type: 'circle'
  })
})

There is a clear relationship, with trip duration strongly correlating linearly with average fare, and this relationship becoming weaker as the trip duration becomes longer. We have clamped the axis to 100 for the average_fare and 130 for the trip_duration to remove anomalous data.

In [0]:
#Query for trip_distance vs. fare amount

%%bigquery --project $project_id v8

SELECT rounded_distance, AVG(total_amount) AS average_fare
FROM  (SELECT ROUND(trip_distance*10)/10 AS rounded_distance, total_amount
      FROM `project-223804.MyTaxiDataset.sample_postweather`)
GROUP BY rounded_distance

In [0]:
alt.Chart(v8).mark_circle(size = 10).encode(
alt.X('rounded_distance', scale = alt.Scale(domain = (0, 40), clamp = True)),
y=alt.Y('average_fare', scale=alt.Scale(type='linear', domain = (0, 110), clamp = True))
)

Chart({
  data:      rounded_distance  average_fare
  0                29.3     60.500000
  1               173.9     70.000000
  2                26.7    106.750000
  3                38.7    183.000000
  4                26.4     73.982000
  5                 4.5     21.635066
  6                10.0     41.853463
  7                 3.2     17.549886
  8                12.3     51.120979
  9                23.0     65.154444
  10               19.5     65.643130
  11               13.3     52.465537
  12               10.5     43.090730
  13               29.9     75.410000
  14                1.8     12.364098
  15                5.5     24.529109
  16                6.0     25.951188
  17               25.7     82.965714
  18               11.7     45.705265
  19                4.0     20.270723
  20               17.5     65.589202
  21               14.9     57.282524
  22                2.2     13.905123
  23               19.1     64.346781
  24                6.5     27.666848
  25               15.1     58.087624
  26                8.1     34.133468
  27                8.7     36.825149
  28               20.7     65.429559
  29               12.7     49.274812
  ..                ...           ...
  308              22.9     71.716000
  309              18.0     65.822239
  310               6.4     27.354958
  311              28.9     78.930000
  312              36.6     58.340000
  313             201.1     58.340000
  314              34.0     58.340000
  315              29.8     93.805000
  316              18.5     65.944400
  317               6.1     26.450077
  318              21.4     69.211250
  319              26.1     78.483333
  320              28.1     55.616667
  321              24.0     70.784286
  322              19.2     66.553497
  323              27.5     77.437143
  324              27.9     79.108000
  325              31.8     62.420000
  326              35.6     70.340000
  327              31.4     99.840000
  328              38.1    111.840000
  329              23.5     74.941538
  330               5.1     23.365312
  331               9.5     40.475301
  332              24.2     79.792222
  333              16.1     61.093986
  334               1.3     10.258466
  335              23.7     69.595556
  336              17.6     66.075677
  337              10.2     42.072106
  
  [338 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      scale: Scale({
        clamp: True,
        domain: (0, 40)
      }),
      shorthand: 'rounded_distance'
    }),
    y: Y({
      scale: Scale({
        clamp: True,
        domain: (0, 110),
        type: 'linear'
      }),
      shorthand: 'average_fare'
    })
  }),
  mark: MarkDef({
    size: 10,
    type: 'circle'
  })
})

It is no surprise that distance is one of the best predictors of the fare paid.

In [0]:
#Query to obtain pickup and dropoff locations

%%bigquery --project $project_id v9

SELECT pickup_longitude, pickup_latitude, 'pickup' as binary
FROM   `project-223804.MyTaxiDataset.sample_postweather`

UNION ALL

SELECT dropoff_longitude, dropoff_latitude, 'dropoff' as binary
FROM   `project-223804.MyTaxiDataset.sample_postweather`

,pickup_longitude,pickup_latitude,binary
0,-73.781250,40.644951,pickup
1,-73.781250,40.645020,pickup
2,-73.781250,40.644981,pickup
3,-73.781250,40.644974,pickup
4,-73.875000,40.774063,pickup
5,-73.875000,40.773861,pickup
6,-73.875000,40.773815,pickup
7,-73.875000,40.774071,pickup
8,-73.921875,40.767052,pickup
9,-73.953125,40.791466,pickup


In [0]:
alt.Chart(v9.sample(4000)).mark_circle(size = 2).encode(
x=alt.X('pickup_longitude', scale=alt.Scale(type='linear', domain = (-74.06, -73.85), clamp = True)),
y=alt.Y('pickup_latitude', scale=alt.Scale(type='linear', domain = (40.65, 40.85), clamp = True)),
    color = alt.Color('binary', scale = alt.Scale(domain = ['pickup', 'dropoff'], range = ['red', 'blue']))
)

Chart({
  data:         pickup_longitude  pickup_latitude   binary
  322432        -73.992722        40.752930   pickup
  492776        -73.969009        40.758110  dropoff
  346213        -73.969353        40.800629   pickup
  567328        -73.992455        40.748444  dropoff
  316869        -73.959824        40.776329   pickup
  135460        -73.933365        40.798958   pickup
  416806        -73.975891        40.748604  dropoff
  405678        -73.972534        40.791676  dropoff
  130507        -73.978531        40.787395   pickup
  238306        -73.966339        40.767849   pickup
  521498        -74.009048        40.736130  dropoff
  476214        -73.963509        40.771065  dropoff
  664913        -73.960854        40.772423  dropoff
  391763        -73.968132        40.800339   pickup
  149710        -73.985130        40.732086   pickup
  295473        -73.968857        40.791054   pickup
  366110        -73.975456        40.733078   pickup
  434606        -73.950439        40.771759  dropoff
  770523        -73.994316        40.766315  dropoff
  59812         -73.862740        40.768711   pickup
  350963        -73.970879        40.764294   pickup
  51458         -74.000618        40.722111   pickup
  385174        -73.981621        40.778549   pickup
  211797        -73.988983        40.743580   pickup
  126458        -73.992935        40.768276   pickup
  4381          -73.954468        40.764217   pickup
  572605        -73.994102        40.750023  dropoff
  159889        -73.957115        40.774651   pickup
  346376        -73.953789        40.788006   pickup
  687923        -73.983925        40.764557  dropoff
  ...                  ...              ...      ...
  524315        -73.947464        40.781715  dropoff
  631491        -73.997223        40.718021  dropoff
  442733        -73.953133        40.785839  dropoff
  703904        -73.989182        40.769482  dropoff
  291505        -73.967506        40.763206   pickup
  314346        -73.959091        40.771660   pickup
  287115        -73.997353        40.763020   pickup
  279510        -74.010292        40.711502   pickup
  584020        -73.982262        40.784798  dropoff
  531917        -73.934402        40.760841  dropoff
  82446         -73.979012        40.744495   pickup
  65102         -73.973640        40.763340   pickup
  315634        -73.975082        40.780209   pickup
  714700        -74.007919        40.712257  dropoff
  345407        -73.984734        40.736477   pickup
  372008        -73.914726        40.743507   pickup
  151193        -74.001183        40.731777   pickup
  767634        -73.993401        40.734406  dropoff
  666573        -73.961342        40.774288  dropoff
  301431        -74.001999        40.727974   pickup
  133135        -73.995010        40.744678   pickup
  285778        -74.012550        40.702267   pickup
  349899        -73.970512        40.783749   pickup
  768272        -73.993584        40.733124  dropoff
  591007        -74.015778        40.677368  dropoff
  662854        -73.991493        40.750061  dropoff
  721799        -73.978867        40.744812  dropoff
  688681        -73.984230        40.779991  dropoff
  621385        -73.978302        40.785889  dropoff
  637852        -74.014923        40.714062  dropoff
  
  [4000 rows x 3 columns],
  encoding: EncodingWithFacet({
    color: Color({
      scale: Scale({
        domain: ['pickup', 'dropoff'],
        range: ['red', 'blue']
      }),
      shorthand: 'binary'
    }),
    x: X({
      scale: Scale({
        clamp: True,
        domain: (-74.06, -73.85),
        type: 'linear'
      }),
      shorthand: 'pickup_longitude'
    }),
    y: Y({
      scale: Scale({
        clamp: True,
        domain: (40.65, 40.85),
        type: 'linear'
      }),
      shorthand: 'pickup_latitude'
    })
  }),
  mark: MarkDef({
    size: 2,
    type: 'circle'
  })
})

There is significant clustering, unsurprisingly, and most pickup and dropoff points are in manhattan (corresponding to the U shaped fgure). This suggests that there may be a significant relationship between the locations and the kind/frequency of trips that are being taken and hence the fare.

In [0]:
#Query to show pickup zone vs average_fare (top 10 and bottom 10)

%%bigquery --project $project_id v10

(SELECT pickup_zone, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY pickup_zone
ORDER BY average_fare DESC
LIMIT 10)

UNION ALL

(SELECT pickup_zone, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY pickup_zone
ORDER BY average_fare
LIMIT 10)

ORDER BY average_fare

In [0]:
alt.Chart(v10).mark_bar().encode(
x=alt.X('pickup_zone', sort=alt.EncodingSortField(field="average_fare", op = 'count', order='ascending')),
y=alt.Y('average_fare')
)

Chart({
  data:                      pickup_zone  average_fare
  0                    Parkchester      3.800000
  1                   Howard Beach      5.800000
  2                Queensboro Hill      5.800000
  3                     Co-Op City      6.300000
  4             Murray Hill-Queens      6.800000
  5                   Coney Island      6.825000
  6            Grymes Hill/Clifton      7.050000
  7         Marine Park/Mill Basin      7.300000
  8                  Rockaway Park      7.300000
  9                 Queens Village      7.550000
  10                     Woodhaven     48.465000
  11     Springfield Gardens North     48.550000
  12                Brighton Beach     52.870000
  13  Flushing Meadows-Corona Park     53.721667
  14                     Gravesend     54.360000
  15                   JFK Airport     55.456439
  16     Springfield Gardens South     59.384444
  17                 South Jamaica     60.828421
  18                  Baisley Park     61.447143
  19                Newark Airport     66.021429,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'pickup_zone',
      sort: EncodingSortField({
        field: 'average_fare',
        op: 'count',
        order: 'ascending'
      })
    }),
    y: Y({
      shorthand: 'average_fare'
    })
  }),
  mark: 'bar'
})

It is clear that the pickup_zone has an extremely strong impact on the amouunt paid. Certain pickup zones have extremely high average fares.

In [0]:
#Query to show dropoff zone vs average_fare (top 10 and bottom 10)

%%bigquery --project $project_id v11

(SELECT dropoff_zone, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY dropoff_zone
ORDER BY average_fare DESC
LIMIT 10)

UNION ALL

(SELECT dropoff_zone, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY dropoff_zone
ORDER BY average_fare
LIMIT 10)

ORDER BY average_fare

In [0]:
alt.Chart(v11).mark_bar().encode(
x=alt.X('dropoff_zone', sort=alt.EncodingSortField(field="average_fare", op = 'count', order='ascending')),
y=alt.Y('average_fare')
)

Chart({
  data:                          dropoff_zone  average_fare
  0               Upper East Side South     11.636041
  1               Upper East Side North     11.849010
  2                            Flatiron     12.362829
  3                            Union Sq     12.455277
  4                            Kips Bay     12.523854
  5                        West Village     12.552948
  6                            Gramercy     12.641468
  7                     Lenox Hill West     12.703210
  8                 Lincoln Square East     12.789992
  9             Greenwich Village North     12.825848
  10           South Beach/Dongan Hills     77.178889
  11                        Great Kills     79.380000
  12             New Dorp/Midland Beach     80.086667
  13                    Mariners Harbor     81.726667
  14                      Arden Heights     88.380000
  15                            Oakwood     91.010000
  16                     Newark Airport     94.815661
  17                  Rossville/Woodrow    109.406000
  18  Eltingville/Annadale/Prince's Bay    111.100000
  19             Charleston/Tottenville    116.780000,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'dropoff_zone',
      sort: EncodingSortField({
        field: 'average_fare',
        op: 'count',
        order: 'ascending'
      })
    }),
    y: Y({
      shorthand: 'average_fare'
    })
  }),
  mark: 'bar'
})

It is clear that the dropoff_zone has an extremely strong impact on the amouunt paid. Certain dropoff zones have extremely high average fares.

In [0]:
#Query to show pickup-dropoff route vs average_fare (top 10 and bottom 10)

%%bigquery --project $project_id v12

(SELECT pickup_dropoff, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY pickup_dropoff
ORDER BY average_fare DESC
LIMIT 10)

UNION ALL

(SELECT pickup_dropoff, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY pickup_dropoff
ORDER BY average_fare
LIMIT 10)

ORDER BY average_fare

In [0]:
alt.Chart(v12).mark_bar().encode(
x=alt.X('pickup_dropoff', sort=alt.EncodingSortField(field="average_fare", op = 'count', order='ascending')),
y=alt.Y('average_fare', scale=alt.Scale(type = 'log'))
)

Chart({
  data:                                        pickup_dropoff  average_fare
  0            Briarwood/Jamaica Hills-Brooklyn Heights      0.300000
  1   Springfield Gardens South-Briarwood/Jamaica Hills      1.800000
  2                               Ocean Hill-Ocean Hill      3.300000
  3                                 Rego Park-Rego Park      3.300000
  4                      East Harlem South-Forest Hills      3.300000
  5                      Union Sq-Van Cortlandt Village      3.300000
  6               Claremont/Bathgate-Claremont/Bathgate      3.300000
  7                             Parkchester-Parkchester      3.800000
  8                      Morningside Heights-Highbridge      3.800000
  9                         Lower East Side-Hunts Point      3.800000
  10                   East Williamsburg-Newark Airport    113.800000
  11                East Chelsea-Charleston/Tottenville    116.780000
  12                     Lenox Hill East-Newark Airport    118.265000
  13                      Murray Hill-Rossville/Woodrow    119.300000
  14                        Battery Park-Newark Airport    122.800000
  15                   LaGuardia Airport-Newark Airport    134.691111
  16                      JFK Airport-Rossville/Woodrow    134.880000
  17                            Steinway-Newark Airport    148.610000
  18                         JFK Airport-Newark Airport    157.111250
  19                    Clinton West-South Williamsburg    427.920000,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'pickup_dropoff',
      sort: EncodingSortField({
        field: 'average_fare',
        op: 'count',
        order: 'ascending'
      })
    }),
    y: Y({
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'average_fare'
    })
  }),
  mark: 'bar'
})

Separating by route, we get even larger differences in average fares. The top value is rather extreme, but it might be explained by small number of data points, with high variance, causing the average to be extremely skewed by a few data points.

In [0]:
#Query to show passenger_count vs average_fare

%%bigquery --project $project_id v13

SELECT passenger_count, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY passenger_count

In [0]:
alt.Chart(v13).mark_bar(size = 30).encode(
x=alt.X('passenger_count'),
y=alt.Y('average_fare')
)

Chart({
  data:    passenger_count  average_fare
  0                1     15.772494
  1                3     16.336769
  2                2     16.588091
  3                5     16.316478
  4                4     16.568450
  5                6     15.936942,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'passenger_count'
    }),
    y: Y({
      shorthand: 'average_fare'
    })
  }),
  mark: MarkDef({
    size: 30,
    type: 'bar'
  })
})

Surprisingly, passenger_count has a very weak effect on fares, with no clear trend.

In [0]:
#Query to show rate_type vs average_fare

%%bigquery --project $project_id v14

SELECT CASE rate_code
    WHEN 1 THEN 'Standard rate'
    WHEN 2 THEN 'JFK'
    WHEN 3 THEN 'Newark'
    WHEN 4 THEN 'Nassau or Westchester'
    WHEN 5 THEN 'Negotiated Fare'
    WHEN 6 THEN 'Group ride'
    ELSE 'INVALID'
    END AS rate_type, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY rate_code

In [0]:
alt.Chart(v14).mark_bar().encode(
x=alt.X('rate_type'),
y=alt.Y('average_fare')
)

Chart({
  data:                rate_type  average_fare
  0          Standard rate     14.790574
  1                    JFK     64.954910
  2        Negotiated Fare     52.047067
  3                 Newark     91.664632
  4  Nassau or Westchester     24.335000
  5             Group ride      6.300000,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'rate_type'
    }),
    y: Y({
      shorthand: 'average_fare'
    })
  }),
  mark: 'bar'
})

Rate type, not surprisingly, has a very strong impact on fares.

In addition, we might hypothesize that prices are driven by demand and supply. That is, higher demand timings/areas might have higher prices. We estimate high demand timings by having different buckets of time and counting the number of trips during that period. Presumably, higher trip count means higher demand.

In [0]:
#Query to show period_count vs average_fare

%%bigquery --project $project_id v15

SELECT COUNT(*) AS period_count, AVG(total_amount) AS average_fare
FROM `project-223804.MyTaxiDataset.sample_postweather`
GROUP BY pickup_month, pickup_day, pickup_hour

In [0]:
alt.Chart(v15).mark_circle(size = 20).encode(
x=alt.X('period_count'),
y=alt.Y('average_fare')
)

Chart({
  data:       period_count  average_fare
  0                1      7.800000
  1                1     36.000000
  2                1      7.800000
  3                1      4.300000
  4                1     27.300000
  5                2      8.505000
  6                2     20.150000
  7                4     17.290000
  8                4     21.800000
  9                5     26.900000
  10               5     16.390000
  11               5     22.960000
  12               5     19.942000
  13               5     12.270000
  14               5     20.632000
  15               6     28.150000
  16               6     11.391667
  17               6     17.665000
  18               6     28.086667
  19               6     16.575000
  20               6     13.530000
  21               6     12.966667
  22               7     30.647143
  23               7     14.014286
  24               7     11.962857
  25               7     31.095714
  26               7      8.822857
  27               7     13.985714
  28               8     14.020000
  29               8     12.517500
  ...            ...           ...
  4327           169     14.996746
  4328           169     14.262071
  4329           170     16.784647
  4330           171     17.228421
  4331           171     16.991287
  4332           171     15.423918
  4333           171     15.179942
  4334           172     16.391221
  4335           172     16.062558
  4336           172     14.328081
  4337           172     15.062849
  4338           172     14.077733
  4339           173     14.931792
  4340           173     14.632543
  4341           174     14.685172
  4342           174     17.163793
  4343           175     15.725200
  4344           175     16.358400
  4345           177     14.633785
  4346           177     16.589040
  4347           177     15.805254
  4348           179     15.871285
  4349           180     15.287000
  4350           181     15.073260
  4351           181     15.474586
  4352           184     17.884946
  4353           184     15.218478
  4354           184     15.316902
  4355           187     15.066631
  4356           196     14.698316
  
  [4357 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'period_count'
    }),
    y: Y({
      shorthand: 'average_fare'
    })
  }),
  mark: MarkDef({
    size: 20,
    type: 'circle'
  })
})

Surprisingly, there is no clear positive relation between period_count and average_fare. Rather, what we observe is that as period_count increases, the variance in average_fare decreases, which makes sense as a larger number of trips means that deviation is reduced.

However, it might be the case that it is local demand that matters; i.e. it is demand in a certain period in a certain area that actually drives up demand. We will thus measure this by grouping by both zone and period, and see if this changes the results.

In [0]:
#Query to show zone_period_count vs average_fare

%%bigquery --project $project_id v16
SELECT zone_period_count, AVG(average_fare) AS avg_avg_fare
FROM  (SELECT COUNT(*) AS zone_period_count, AVG(total_amount) AS average_fare
      FROM `project-223804.MyTaxiDataset.sample_postweather`
      GROUP BY pickup_month, pickup_day, pickup_hour, pickup_zone)
GROUP BY zone_period_count

In [0]:
alt.Chart(v16).mark_bar().encode(
x=alt.X('zone_period_count'),
y=alt.Y('avg_avg_fare')
)

Chart({
  data:     zone_period_count  avg_avg_fare
  0                   1     16.292039
  1                   6     15.862994
  2                   3     16.089974
  3                   9     15.181327
  4                   5     16.094761
  5                   7     15.480381
  6                   4     16.041252
  7                   2     15.956546
  8                   8     15.360407
  9                  10     15.146026
  10                 12     14.792425
  11                 11     14.405672
  12                 13     14.202943
  13                 14     14.877927
  14                 15     14.966613
  15                 17     15.535714
  16                 16     14.366591
  17                 18     14.527407
  18                 19     16.409474
  19                 22     16.313409,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'zone_period_count'
    }),
    y: Y({
      shorthand: 'avg_avg_fare'
    })
  }),
  mark: 'bar'
})

Zone_period_count has a weak relationship to fare. We notice that extremes tend to have higher fare than the middle values. One possible explanation of this data is this: a time+place with very low trip count might indicate an inaccessible/difficult to service place with generally higher fees, but it's inaccessibility disincentivizes services. A time+place with very high trip count might indicate such high demand that prices actually increase.

Now we will study weather effects

In [0]:
#Query to show pickup_temperature vs average_fare

%%bigquery --project $project_id v17
SELECT rounded_temp, AVG(total_amount) AS avg_fare, COUNT(*) AS count
FROM
    (SELECT *, ROUND(pickup_temp,-1) AS rounded_temp
    FROM  `project-223804.MyTaxiDataset.sample_postweather`)
GROUP BY rounded_temp

,rounded_temp,avg_fare,count
0,10.0,14.569524,2164
1,20.0,15.142930,11218
2,30.0,15.333331,50369
3,40.0,15.602171,81177
4,50.0,16.185685,100361
5,60.0,16.310855,71624
6,70.0,16.355872,60881
7,80.0,16.103102,15930


In [0]:
alt.Chart(v17).mark_bar().encode(
x=alt.X('rounded_temp'),
y=alt.Y('avg_fare')
)

Chart({
  data:    rounded_temp   avg_fare   count
  0          10.0  14.569524    2164
  1          20.0  15.142930   11218
  2          30.0  15.333331   50369
  3          40.0  15.602171   81177
  4          50.0  16.185685  100361
  5          60.0  16.310855   71624
  6          70.0  16.355872   60881
  7          80.0  16.103102   15930,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'rounded_temp'
    }),
    y: Y({
      shorthand: 'avg_fare'
    })
  }),
  mark: 'bar'
})

In [0]:
#Query to show pickup_precipitation vs average_fare

%%bigquery --project $project_id v18
SELECT rounded_prcp, AVG(total_amount) AS avg_fare, COUNT(*) AS count
FROM
    (SELECT *, ROUND(pickup_prcp,2) AS rounded_prcp
    FROM  `project-223804.MyTaxiDataset.sample_postweather`)
GROUP BY rounded_prcp
HAVING COUNT(*) > 5

In [0]:
alt.Chart(v18).mark_bar().encode(
x=alt.X('rounded_prcp'),
y=alt.Y('avg_fare')
)

Chart({
  data:     rounded_prcp   avg_fare   count
  0           0.00  16.035358  241218
  1           0.03  15.153877   14662
  2           0.01  15.183657   40591
  3          33.33  42.504762     105
  4           0.02  16.193289    9164
  5           0.04  16.390148   11444
  6           0.05  44.966262     543
  7           0.07  15.054026   12283
  8           0.23  15.205742    6214
  9           0.21  15.858521    2198
  10          0.17  40.096036     111
  11          0.14  14.244427    2094
  12          0.36  14.483231    2083
  13          0.38  42.651471     272
  14          0.13  14.227951    4256
  15          0.20  18.722828    2012
  16          0.09  16.045051    6551
  17          0.06  16.137537    4230
  18          0.43  43.510000     123
  19          0.52  13.410599    1771
  20          0.56  14.888952    2119
  21          0.19  13.671965    2260
  22          0.12  16.746341    4599
  23          0.29  14.615159    2109
  24          0.46  16.901243    1979
  25          0.42  46.473459     133
  26          0.49  14.161431    2026
  27          0.82  16.501006    1431
  28          0.86  40.433841     164
  29          0.27  16.280301    1996
  30          0.44  33.636842      57
  31          0.91  13.453959     783
  32          0.24   8.800000       7
  33          0.15  15.587468    2433
  34          0.30  42.075040     125
  35          0.10  14.086658    4297
  36          0.18  42.175233     258
  37          0.11  41.130298     369
  38          0.08  13.780119    1852
  39          0.48  37.000574     122
  40          0.39  13.290769    1833
  41          0.45  45.377529     170
  42          0.25  14.276010     411
  43          0.37  46.394485     136
  44          0.26  42.719533     107,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'rounded_prcp'
    }),
    y: Y({
      shorthand: 'avg_fare'
    })
  }),
  mark: 'bar'
})

There data seems to split into two buckets, very low precipitations (where there is presumably no rain), and a high precipitation of 33, indicating rain. There is likely to be some kind of relationship, but it seems that the weather data is too erratic and sparse for any relationship to be drawn.

In [0]:
#Query to show pickup snowdepth vs average_fare

%%bigquery --project $project_id v19
SELECT rounded_sndp, AVG(total_amount) AS avg_fare, COUNT(*) AS count
FROM
    (SELECT *, ROUND(pickup_sndp,0) AS rounded_sndp
    FROM  `project-223804.MyTaxiDataset.sample_postweather`)
GROUP BY rounded_sndp
HAVING COUNT(*) > 200

,rounded_sndp,avg_fare,count
0,0.0,15.880100,371029
1,1.0,20.796457,5623
2,3.0,14.726149,7369
3,9.0,18.116945,1810
4,11.0,15.246034,1422
5,2.0,17.149704,2029
6,5.0,16.332674,2180
7,8.0,14.878332,1919


In [0]:
alt.Chart(v19).mark_bar().encode(
x=alt.X('rounded_sndp'),
y=alt.Y('avg_fare')
)

Chart({
  data:    rounded_sndp   avg_fare   count
  0           0.0  15.880100  371029
  1           1.0  20.796457    5623
  2           3.0  14.726149    7369
  3           9.0  18.116945    1810
  4          11.0  15.246034    1422
  5           2.0  17.149704    2029
  6           5.0  16.332674    2180
  7           8.0  14.878332    1919,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'rounded_sndp'
    }),
    y: Y({
      shorthand: 'avg_fare'
    })
  }),
  mark: 'bar'
})

There does not seem to be a consistent pattern being presented from the snow depth data. Like the rain data, it might be that the data provided from the weather tables are too sparse and erratic for any relationships to be shown.

Overall, for this section, the strongest relationship is between price and temperature. Higher temperatures mean people demand more cabs; a pattern that could evolve across seasons, across different times of the day, etc. It 

In [6]:
model_dataset_name = 'rideml'

dataset = bigquery.Dataset(client.dataset(model_dataset_name))
client.create_dataset(dataset)

In [0]:
CREATE OR REPLACE MODEL `project-223804.rideml.ride_model`
OPTIONS
  ( model_type='linear_reg',
    ls_init_learn_rate=.15,
    l1_reg=1,
    max_iterations=5) AS
SELECT CAST(EXTRACT(HOUR FROM training.pickup_hour) AS STRING) AS feature_1,
       CAST(training.pickup_dayofweek AS STRING) AS feature_2,
       CAST(training.pickup_day AS STRING) AS feature_3,
       CAST(training.pickup_month AS STRING) AS feature_4,
       CAST(training.trip_duration AS STRING) AS feature_5,
       training.trip_distance AS feature_6,
       training.pickup_zone AS feature_7,
       training.dropoff_zone AS feature_8,
       training.pickup_dropoff AS feature_9,
       training.passenger_count AS feature_10,
       training.rate_code AS feature_11,
       ROUND(training.pickup_temp, -1) AS feature_12,
       training.total_amount AS label
FROM `project-223804.MyTaxiDataset.training_2016` AS training

In [13]:
%%bigquery --project $project_id

SELECT *
FROM ML.EVALUATE(MODEL `project-223804.rideml.ride_model`, (
    SELECT CAST(EXTRACT(HOUR FROM training.pickup_hour) AS STRING) AS feature_1,
       CAST(training.pickup_dayofweek AS STRING) AS feature_2,
       CAST(training.pickup_day AS STRING) AS feature_3,
       CAST(training.pickup_month AS STRING) AS feature_4,
       CAST(training.trip_duration AS STRING) AS feature_5,
       training.trip_distance AS feature_6,
       training.pickup_zone AS feature_7,
       training.dropoff_zone AS feature_8,
       training.pickup_dropoff AS feature_9,
       training.passenger_count AS feature_10,
       training.rate_code AS feature_11,
       ROUND(training.pickup_temp, -1) AS feature_12,
       training.total_amount AS label
    FROM `project-223804.MyTaxiDataset.evaluation_2016` AS training
  ))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.809866,17.231514,0.022916,1.237256,0.894381,0.895003


We have used linear regression across the features that we have discussed earlier. This is a reasonably good model to predict fare prices as the r-value is 0.89 and close to 1, suggesting a strong positive correlation. The mean_absolute error is also low at 1.81 (where fare prices are approximately at least around 10 times of that). This also suggests that this is a good model.

In [15]:
%%bigquery --project $project_id

SELECT
  *
FROM
  ML.EVALUATE(MODEL `project-223804.rideml.ride_model`, (
    SELECT CAST(EXTRACT(HOUR FROM training.pickup_hour) AS STRING) AS feature_1,
       CAST(training.pickup_dayofweek AS STRING) AS feature_2,
       CAST(training.pickup_day AS STRING) AS feature_3,
       CAST(training.pickup_month AS STRING) AS feature_4,
       CAST(training.trip_duration AS STRING) AS feature_5,
       training.trip_distance AS feature_6,
       training.pickup_zone AS feature_7,
       training.dropoff_zone AS feature_8,
       training.pickup_dropoff AS feature_9,
       training.passenger_count AS feature_10,
       training.rate_code AS feature_11,
       ROUND(training.pickup_temp, -1) AS feature_12,
       training.total_amount AS label
    FROM `project-223804.MyTaxiDataset.prediction_2016` AS training
  ))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.813288,10.726075,0.023918,1.244564,0.933468,0.934096


In [0]:
CREATE OR REPLACE MODEL `project-223804.rideml.ride_model_without_distance`
OPTIONS
  ( model_type='linear_reg',
    ls_init_learn_rate=.15,
    l1_reg=1,
    max_iterations=5) AS
SELECT CAST(EXTRACT(HOUR FROM training.pickup_hour) AS STRING) AS feature_1,
       CAST(training.pickup_dayofweek AS STRING) AS feature_2,
       CAST(training.pickup_day AS STRING) AS feature_3,
       CAST(training.pickup_month AS STRING) AS feature_4,
       CAST(training.trip_duration AS STRING) AS feature_5,
       training.pickup_zone AS feature_7,
       training.dropoff_zone AS feature_8,
       training.pickup_dropoff AS feature_9,
       training.passenger_count AS feature_10,
       training.rate_code AS feature_11,
       ROUND(training.pickup_temp, -1) AS feature_12,
       training.total_amount AS label
FROM `project-223804.MyTaxiDataset.training_2016` AS training

In [17]:
%%bigquery --project $project_id

SELECT *
FROM ML.EVALUATE(MODEL `project-223804.rideml.ride_model_without_distance`, (
    SELECT CAST(EXTRACT(HOUR FROM training.pickup_hour) AS STRING) AS feature_1,
       CAST(training.pickup_dayofweek AS STRING) AS feature_2,
       CAST(training.pickup_day AS STRING) AS feature_3,
       CAST(training.pickup_month AS STRING) AS feature_4,
       CAST(training.trip_duration AS STRING) AS feature_5,
       training.pickup_zone AS feature_7,
       training.dropoff_zone AS feature_8,
       training.pickup_dropoff AS feature_9,
       training.passenger_count AS feature_10,
       training.rate_code AS feature_11,
       ROUND(training.pickup_temp, -1) AS feature_12,
       training.total_amount AS label
    FROM `project-223804.MyTaxiDataset.evaluation_2016` AS training
  ))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,2.026407,16.837582,0.029711,1.380235,0.896796,0.897279


The correlation is lower, but not much lower. This is surprising, and seems to suggest that we have other factors that are highly correlated/approximate the factor of distance. Some hypotheses on what these could be is trip_duration and start zone-end zone.

The final correlation value for our prediction set is also high at 0.93 suggesting a highly positive linear correlation.

Learnings:
Firstly, we learnt the value of using data visualizations to identify possible predictive factors for our model. Our initial querying and visualization, such as comparing trip_distnace vs avg_fare, allowed us to identify very strong relationships which we leveraged to build a successful mode. Furthermore, it allows us to identify what might intuitively be a good predictive factor but turns out to not be (for example, period and zone-period counts). This is important because while including such factors might lead to overfitting on the training data, which leads to lower accuracy on new datasets.

Secondly, we also learnt the value of feature engineering for both visualizing and possibly prediction. We were able to engineer (within a single dataset and across multiple tables), features such as pickup and dropoff zone and start-end routes that were surprisingly strong predictors of our result. Furthermore, looking across different datasets, we learnt how to creatively engineer new features, such as the weather features, some of which turned out to have predictive qualities (e.g. tempeature)

Thirdly, we also learnt about the limits of feature engineering, particularly across different tables. For example, the snow and rain data, while present, was extremely scattered, primarily due to the limitations of the base data itself and loss of data in the dataset combination process.

A thing we did not expect to see is that removing distance did not significantly affect the accuracy of our model. This suggests that we have captured that factor through the other factors.

What has been problematic in our dataset is month and year relations, and certain anomalies we have not been able to explain very well. Due to the limitations of the taxi dataset, the 2017 data being very unclean, and the lack of years in general for us to conduct any study on this feature, we had to omit year as a factor. Furthermore, the 2015 dataset was limited because the first six months had no rate_code entries at all, while for 2016 dataset, the last 6 months have no location entries. We decided that we wanted to study these features, and hence only studied data from the first 6 months of 2016, which prevented us from studying the full range of months. Hence, our model is limited to an analysis of the first 6 months only. 

What we would do differently is to construct a ML model on a different dataset. We constructed one, called MyTaxiDataset2, with the following tables:
evaluation_2015
prediction_2015
prepartition_2015
sample_postweather_2015
sample_preweather_2015
training_2015
weather_borough_period_2015
yellowtrips_2015_cleaned
This basically omits rate data but allows us to capture a full year of 2015. We would like to use this to more closely study month relationship with fare.

A last thing we might do is to isolate some of our features, meticulously examining the exact effects of each of them on the accuracy, and also explore visualizations of their correlations with one another.